In [1]:
from __future__ import absolute_import, division, print_function

import tensorflow as tf



In [2]:
# anulo el uso de la GPU por falta de memoria
import os, sys
os.environ['CUDA_VISIBLE_DEVICES'] = '1' #'-1' for no GPU

if tf.test.gpu_device_name():
    print('GPU found')
else:
    print("No GPU found")

# No GPU found

# tf.enable_eager_execution()
tf.VERSION

No GPU found


'1.13.1'

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import skimage.io
from skimage.transform import resize
from imgaug import augmenters as iaa
from tqdm import tqdm
import PIL
from PIL import Image, ImageOps
import cv2

from sklearn.utils import class_weight, shuffle
from keras.losses import binary_crossentropy
from keras.applications.resnet50 import preprocess_input
from keras.preprocessing import image
from keras.models import Sequential
from keras.layers import InputLayer
from keras.layers import Dense
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.models import load_model
import keras.backend as K

from sklearn.metrics import f1_score, fbeta_score
from keras.utils import Sequence
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split

WORKERS = 2
CHANNEL = 3

import warnings
warnings.filterwarnings("ignore")
IMG_SIZE = 512
NUM_CLASSES = 5
SEED = 77
TRAIN_NUM = 1000 # use 1000 when you just want to explore new idea, use -1 for full train


DATA_PATH = "data"

Using TensorFlow backend.


In [4]:
import os
print(os.listdir("data"))

['results_df_20190710232755.pkl', 'results_test_df_20190710232755.pkl', 'processed_train_images', 'train.csv', 'results_df.pkl', 'train_images', 'test.csv', 'test_images', 'processed_test_images', 'results_test_df.pkl', 'sample_submission.csv']


In [5]:
df_train = pd.read_csv(DATA_PATH + '/train.csv')
df_test = pd.read_csv(DATA_PATH + '/test.csv')

# The Model
## inspired in real life

help obtained from
https://machinelearningmastery.com/keras-functional-api-deep-learning/


In [6]:
# Aquí transcribimos las imágenes a un fichero tipo TFRECORD de tamaño
width = 100 # 400
height = 125 # 500

size = (width,height)

In [7]:
# ---------------------
# OPTION 2
# ---------------------

''' '''


model = Sequential()

model.add(InputLayer((width,height,1)))
model.add(Conv2D(1000, kernel_size=(5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(5, 5)))
model.add(Conv2D(500, kernel_size=(2, 2), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(100, kernel_size=(2, 2), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(10*NUM_CLASSES, activation='relu'))
model.add(Dense(NUM_CLASSES, activation='softmax'))


Instructions for updating:
Colocations handled automatically by placer.


In [8]:
model_file = "/mnt/DATA-SSD/DataSandbox/model_2_0100.h5"
model.load_weights(model_file)

In [9]:
# import pydot
# import graphviz

print(model.summary())
# no va por alguna chuminada
# tf.keras.utils.plot_model(model, 'model.png')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 96, 121, 1000)     26000     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 19, 24, 1000)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 18, 23, 500)       2000500   
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 9, 11, 500)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 8, 10, 100)        200100    
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 4, 5, 100)         0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 2000)              0         
__________

In [10]:
def diagnoses_from_list(row):
    n = 0.0
    diagnoses = 0.0
    for i in row[0]:
        diagnoses += i*n
        n += 1.0
    return int(diagnoses)

def crop_image1(img,tol=8):
    # img is image data
    # tol  is tolerance
        
    mask = img>tol
    return img[np.ix_(mask.any(1),mask.any(0))]


In [12]:
results_test = {}
for i in df_test.iterrows():
    path=f"{DATA_PATH}/test_images/{i[1][0]}.png"
    img = cv2.imread(path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    img = crop_image1(img)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(6,6))
    img = clahe.apply(img)
    img = cv2.resize(img, size)
    
    img = image.img_to_array(img)
    img = img.reshape(1,width,height,1)

    results_test[i[0]] = [i[1][0], model.predict(img)]

In [13]:
results_test_df = pd.DataFrame.from_dict(results_test, 
                                    orient='index', 
                                    columns=['id_code', 'prediction'])

In [14]:
# print(results_df.iloc[0][2])
# print(diagnoses_from_list(results_df.prediction[0]))
results_test_df['diagnosis'] = results_test_df.apply( 
    lambda x: diagnoses_from_list(x.prediction), axis=1)
# results_df

In [15]:
results_test_df.drop('prediction',axis=1)
results_test_df.head(10)

,id_code,prediction,diagnosis
0,0005cfc8afb6,"[[0.0, 0.0, 0.0, 1.0, 0.0]]",3
1,003f0afdcd15,"[[0.0, 0.0, 0.0, 1.0, 0.0]]",3
2,006efc72b638,"[[0.0, 0.0, 0.0, 0.0, 1.0]]",4
3,00836aaacf06,"[[0.0, 0.0, 1.0, 0.0, 0.0]]",2
4,009245722fa4,"[[0.0, 0.0, 0.0, 0.0, 1.0]]",4
5,009c019a7309,"[[0.0, 0.0, 0.0, 1.0, 0.0]]",3
6,010d915e229a,"[[0.0, 0.0, 1.0, 0.0, 0.0]]",2
7,0111b949947e,"[[0.0, 0.0, 0.0, 0.0, 1.0]]",4
8,01499815e469,"[[0.0, 0.0, 1.0, 0.0, 0.0]]",2
9,0167076e7089,"[[0.0, 0.0, 1.0, 0.0, 0.0]]",2


In [17]:
results_test_df.to_csv('submission.csv', index=False )